In [1]:
import pandas as pd
import numpy as np
import datetime
import csv

#%matplotlib inline

# Leo csv

In [2]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [3]:
#Es lo que uso (mediante merge con cada grupo de csv) para quedarme solo con datos que están en test_final_100k
postulantesDelTest = test_final.drop_duplicates(subset=['idpostulante'],keep='last')
avisosDelTest  = test_final.drop_duplicates(subset=['idaviso'],keep='last')

postulantesDelTest = postulantesDelTest.drop('idaviso',axis=1)
avisosDelTest  =  avisosDelTest.drop('idpostulante',axis=1)

In [4]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [5]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [6]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

In [7]:
vistas = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index().drop('index',axis=1)

# Borro duplicados y agrego edad

In [8]:
postulantes_genero_y_edad = postulantes_genero_y_edad.merge(postulantesDelTest,on='idpostulante',how='inner')

postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [9]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [10]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(['fechanacimiento'],axis=1)

In [11]:
# En un principio los relleno con la media de los postulantes
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int32')

In [12]:
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [13]:
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)

avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [14]:
postulaciones = postulaciones.drop_duplicates(keep='last')
postulaciones = postulaciones.reset_index().drop('index',axis=1)

postulaciones = postulaciones.merge(postulantesDelTest,on='idpostulante',how='inner')

In [15]:
#Se me ocurrió que un postulante pudo ver un aviso mas de una vez.
#Creo  que con lo de abajo limpio todos pero no estoy seguro.
vistas = vistas.drop_duplicates(subset=['idAviso','idpostulante'],keep='last')
vistas = vistas.reset_index().drop('index',axis=1)

vistas = vistas.rename(columns={'idAviso': 'idaviso'})

In [16]:
vistas = vistas.merge(postulantesDelTest,on='idpostulante',how='inner')
vistas = vistas.merge(avisosDelTest,on='idaviso',how='inner')

# Tasas de postulacion segun idaviso y idpostulante

In [17]:
# paso a datetime
postulaciones['fechapostulacion'] = pd.to_datetime(postulaciones['fechapostulacion'])
postulaciones['fechapostulacion'] = postulaciones['fechapostulacion'].dt.date
vistas['timestamp'] = pd.to_datetime(vistas['timestamp'])
vistas['timestamp'] = vistas['timestamp'].dt.date

In [18]:
# tasa de postulaciones segun aviso
postulaciones_validas = postulaciones[postulaciones['fechapostulacion'].isin(vistas['timestamp'].drop_duplicates())]
vistas_validas = vistas[vistas['timestamp'].isin(postulaciones_validas['fechapostulacion'].drop_duplicates())]

postulaciones_validas = postulaciones_validas.groupby('idaviso').agg({'idpostulante':'count'})
postulaciones_validas = postulaciones_validas.reset_index()
vistas_validas = vistas_validas.groupby('idaviso').agg({'idpostulante':'count'})
vistas_validas = vistas_validas.reset_index()

tasa_idaviso = postulaciones_validas.merge(vistas_validas,on='idaviso')
tasa_idaviso['tasa_idaviso'] = tasa_idaviso['idpostulante_x'] / tasa_idaviso['idpostulante_y']
tasa_idaviso = tasa_idaviso.drop(columns=['idpostulante_x','idpostulante_y'])

In [19]:
# tasa de postulaciones segun postulante
postulaciones_validas = postulaciones[postulaciones['fechapostulacion'].isin(vistas['timestamp'].drop_duplicates())]
vistas_validas = vistas[vistas['timestamp'].isin(postulaciones_validas['fechapostulacion'].drop_duplicates())]

postulaciones_validas = postulaciones_validas.groupby('idpostulante').agg({'idaviso':'count'})
postulaciones_validas = postulaciones_validas.reset_index()
vistas_validas = vistas_validas.groupby('idpostulante').agg({'idaviso':'count'})
vistas_validas = vistas_validas.reset_index()

tasa_idpostulante = postulaciones_validas.merge(vistas_validas,on='idpostulante')
tasa_idpostulante['tasa_idpostulante'] = tasa_idpostulante['idaviso_x'] / tasa_idpostulante['idaviso_y']
tasa_idpostulante = tasa_idpostulante.drop(columns=['idaviso_x','idaviso_y'])

In [20]:
postulantes_genero_y_edad = postulantes_genero_y_edad.merge(tasa_idpostulante,how='left',on='idpostulante')
postulantes_genero_y_edad = postulantes_genero_y_edad.fillna(0.5)

In [21]:
vistas = vistas.merge(tasa_idaviso,how='left',on='idaviso')
vistas = vistas.fillna(0.5)

# Funciones

In [22]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [23]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [24]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'sepostulo'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [25]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

# Tupla no postulada

In [26]:
gente_postulada = postulaciones
gente_postulada = gente_postulada.drop_duplicates(subset=['idpostulante'],keep='last')
gente_postulada = gente_postulada.reset_index().drop(['index','fechapostulacion'],axis=1)

In [27]:
gente_postulada.head()

,idaviso,idpostulante
0,1112094914,exNxxr
1,1112451206,1LXdVB
2,1112412656,6PWG4v
3,1112460141,eOk3oo
4,1112339122,5P9lqk


In [28]:
gente_postulada.shape

(46859, 2)

In [29]:
#Armo la tupla no postulada con gente que se postuló pero asociada a avisos con los que no se postuló.

#Primero hago un sample de gente postulada contra todos los avisos.
#Luego hago un merge con las postulaciones. Los que tengan fechapostulacion en NaN no están postulados.
tupla_no_postulada = generador_de_sample_serie(avisos_detalle['idaviso'],gente_postulada['idpostulante'],100,134)
tupla_no_postulada = tupla_no_postulada.merge(postulaciones,how='left',on=['idaviso','idpostulante'])

tupla_no_postulada = tupla_no_postulada.fillna(0)
tupla_no_postulada = tupla_no_postulada.loc[tupla_no_postulada.fechapostulacion==0]
tupla_no_postulada = tupla_no_postulada.rename(columns={'fechapostulacion': 'y'})

In [30]:
cantidadDeNoPostulados = tupla_no_postulada.shape[0]
cantidadDeNoPostulados

1338559

In [31]:
tupla_no_postulada = tupla_no_postulada.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
tupla_no_postulada = oneHotEncoding(tupla_no_postulada,'sexo')
tupla_no_postulada = change_column_order(tupla_no_postulada, 'y', 0)

In [32]:
tupla_no_postulada.head()

,y,idaviso,idpostulante,edad,tasa_idpostulante,FEM,MASC,NO_DECLARA
0,0,1112390041,66G584,46,0.636364,1,0,0
1,0,1112390041,NRJlDx,37,1.000000,0,1,0
2,0,1112390041,akjGwWJ,24,0.500000,0,1,0
3,0,1112390041,aGbYA9,31,0.552632,1,0,0
4,0,1112390041,Rz6V2QY,26,0.428571,0,1,0


# Tupla postulada

In [33]:
tupla_postulada = postulaciones.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
tupla_postulada = tupla_postulada.drop('fechapostulacion',axis=1)
tupla_postulada = oneHotEncoding(tupla_postulada,'sexo')
tupla_postulada['y'] = 1
tupla_postulada = change_column_order(tupla_postulada, 'y', 0)

In [34]:
tupla_postulada.head()

,y,idaviso,idpostulante,edad,tasa_idpostulante,FEM,MASC,NO_DECLARA
0,1,1112305215,exNxxr,34,0.5,0,1,0
1,1,1112286277,exNxxr,34,0.5,0,1,0
2,1,1112275246,exNxxr,34,0.5,0,1,0
3,1,1112265392,exNxxr,34,0.5,0,1,0
4,1,1112315858,exNxxr,34,0.5,0,1,0


In [35]:
tupla_postulada.shape

(1344469, 8)

# Tupla final para entrenar, con feature "visto"

In [36]:
tuplas = tupla_no_postulada.append(tupla_postulada.sample(n=cantidadDeNoPostulados,random_state=0))

In [37]:
tuplas.head()

,y,idaviso,idpostulante,edad,tasa_idpostulante,FEM,MASC,NO_DECLARA
0,0,1112390041,66G584,46,0.636364,1,0,0
1,0,1112390041,NRJlDx,37,1.000000,0,1,0
2,0,1112390041,akjGwWJ,24,0.500000,0,1,0
3,0,1112390041,aGbYA9,31,0.552632,1,0,0
4,0,1112390041,Rz6V2QY,26,0.428571,0,1,0


In [38]:
tuplas.tail()

,y,idaviso,idpostulante,edad,tasa_idpostulante,FEM,MASC,NO_DECLARA
860171,1,1112335159,vVjGva5,19,5.958333,0,1,0
245649,1,1112437070,vVjAWwQ,30,7.833333,1,0,0
1315496,1,1112277683,Z1MbGZ,28,0.515789,0,1,0
684108,1,1112297028,mzVAOAZ,22,0.902439,1,0,0
480832,1,1112410167,aExN6N,29,0.727273,1,0,0


In [39]:
vistas['visto'] = 1
vistas = vistas.drop('timestamp',axis=1)

In [40]:
#Esto es lo que yo hago para ver que tuplas del sample están en vistas: Hago un merge sobre dos columnas.
#Las que dan NaN es porque no están, entonces las pongo en cero. Las que estan quedan en 1 porque asi
#estaba la columna de visto. Pueden probar el isin de ustedes si les gusta mas.
tuplas = tuplas.merge(vistas,how='left',on=['idaviso','idpostulante'])

In [41]:
tuplas.isnull().sum()

y                          0
idaviso                    0
idpostulante               0
edad                       0
tasa_idpostulante          0
FEM                        0
MASC                       0
NO_DECLARA                 0
tasa_idaviso         2289428
visto                2289428
dtype: int64

In [42]:
tuplas['visto'] = tuplas['visto'].fillna(0)

In [43]:
tuplas.visto.value_counts()

0.0    2289428
1.0     387690
Name: visto, dtype: int64

In [44]:
tuplas['tasa_idaviso'] = tuplas['tasa_idaviso'].fillna(0.5)

In [45]:
tuplas = tuplas.drop(['idaviso','idpostulante'],axis=1)

In [46]:
tuplas.head()

,y,edad,tasa_idpostulante,FEM,MASC,NO_DECLARA,tasa_idaviso,visto
0,0,46,0.636364,1,0,0,0.5,0.0
1,0,37,1.000000,0,1,0,0.5,0.0
2,0,24,0.500000,0,1,0,0.5,0.0
3,0,31,0.552632,1,0,0,0.5,0.0
4,0,26,0.428571,0,1,0,0.5,0.0


# Test final

In [47]:
test_final = test_final.merge(postulantes_genero_y_edad,on='idpostulante',how='left')

In [48]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [49]:
test_final['edad'] = test_final['edad'].astype('int32')
test_final = oneHotEncoding(test_final,'sexo')

In [50]:
test_final.head()

,idaviso,idpostulante,edad,tasa_idpostulante,FEM,MASC,NO_DECLARA
0,739260,6M9ZQR,42,0.5,1,0,0
1,739260,6v1xdL,31,0.5,0,1,0
2,739260,ezRKm9,36,1.0,1,0,0
3,758580,1Q35ej,69,0.5,0,1,0
4,758580,EAN4J6,32,1.0,1,0,0


In [51]:
#Lo mismo que hice con el sample antes lo hago con el test_final
test_final = test_final.merge(vistas,how='left',on=['idaviso','idpostulante'])

In [52]:
test_final['visto'] = test_final['visto'].fillna(0)

In [53]:
test_final.visto.value_counts()

0.0    65839
1.0    34161
Name: visto, dtype: int64

In [54]:
test_final['tasa_idaviso'] = test_final['tasa_idaviso'].fillna(0.5)

In [55]:
test_final = test_final.drop(['idaviso','idpostulante'],axis=1)

In [56]:
test_final.head()

,edad,tasa_idpostulante,FEM,MASC,NO_DECLARA,tasa_idaviso,visto
0,42,0.5,1,0,0,0.5,0.0
1,31,0.5,0,1,0,0.5,0.0
2,36,1.0,1,0,0,0.5,0.0
3,69,0.5,0,1,0,0.5,0.0
4,32,1.0,1,0,0,0.5,0.0


In [57]:
train_label = []
train_data = []

train_label = tuplas.iloc[:, 0]
train_data = tuplas.iloc[:, tuplas.columns != 'y']

# Algoritmos de machine learning

In [58]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [59]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [61]:
ModeloRFC.fit(train_data, train_label.astype('int8'))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [62]:
result = ModeloRFC.predict(test_final)

In [63]:
result.size

100000

In [66]:
write_submission("../data/test_final_100k.csv", result, "predicciones/RFC_Alternativa.csv")

In [67]:
prueba = pd.read_csv('predicciones/RFC_Alternativa.csv')

In [68]:
prueba.sepostulo.value_counts()

1.0    52036
0.0    47964
Name: sepostulo, dtype: int64

In [ ]:
del ModeloRFC

In [ ]:
#-----------------------------------------------------------------
# Modelo GradientBoostingClassifier
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
modeloGBC=GradientBoostingClassifier(n_estimators=150)

In [ ]:
modeloGBC.fit(train_data, train_label)

In [ ]:
result = modeloGBC.predict(test_final)

In [ ]:
result.size

In [ ]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/GBC_AlternativaV7.csv")

In [ ]:
prueba = pd.read_csv('../predicciones/GBC_AlternativaV7.csv')

In [ ]:
prueba.sepostulo.value_counts()

# santi

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
train_data, X_test, train_label, y_test = train_test_split(train_data,train_label,random_state=0,test_size=0.01)

In [ ]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [ ]:
ModeloRFC.fit(train_data, train_label.astype('int8'))

In [ ]:
result = ModeloRFC.predict(X_test)

In [ ]:
roc_auc_score(y_test.astype('int8'),result)